In [7]:
import gensim
import numpy as np
import multiprocessing
import glob
import re
import nltk
import io
import matplotlib.pyplot as plt
from pprint import pprint
%pylab inline
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

Populating the interactive namespace from numpy and matplotlib


In [8]:
book_names = sorted(glob.glob('final_data/*.txt'))
pprint(book_names)

['final_data/book1.txt',
 'final_data/book2.txt',
 'final_data/book3.txt',
 'final_data/book4.txt',
 'final_data/book5.txt',
 'final_data/book6.txt',
 'final_data/book7.txt']


In [14]:
potter_corpus = u''
for book_name in book_names:
    with io.open(book_name, 'r', encoding='utf-8') as file:
        potter_corpus += file.read()
    potter_corpus += ' '
pprint(potter_corpus[:500])

('the boy who lived mr. and mrs. dursley of number four privet drive were '
 'proud to say that they were perfectly normal thank you very much. they were '
 'the last people you’d expect to be involved in anything strange or '
 'mysterious because they just didn’t hold with such nonsense. mr. dursley was '
 'the director of a firm called grunnings which made drills. he was a big '
 'beefy man with hardly any neck although he did have a very large mustache. '
 'mrs. dursley was thin and blonde and had nearly twice the usu')


In [16]:
pprint(len(potter_corpus))

6164634


In [17]:
sentences = nltk.sent_tokenize(potter_corpus)

In [21]:
sentences[:5]

['the boy who lived mr. and mrs. dursley of number four privet drive were proud to say that they were perfectly normal thank you very much.',
 'they were the last people you’d expect to be involved in anything strange or mysterious because they just didn’t hold with such nonsense.',
 'mr. dursley was the director of a firm called grunnings which made drills.',
 'he was a big beefy man with hardly any neck although he did have a very large mustache.',
 'mrs. dursley was thin and blonde and had nearly twice the usual amount of neck which came in very useful as she spent so much of her time craning over garden fences spying on the neighbors.']

In [37]:
def wordlist(raw):
    clean = re.sub("[^a-zA-Z]"," ", raw)
    words = clean.split()
    return words

In [38]:
split_sentences = []
for sent in sentences:
    split_sentences.append(wordlist(sent))

In [39]:
split_sentences[1]

['they',
 'were',
 'the',
 'last',
 'people',
 'you',
 'd',
 'expect',
 'to',
 'be',
 'involved',
 'in',
 'anything',
 'strange',
 'or',
 'mysterious',
 'because',
 'they',
 'just',
 'didn',
 't',
 'hold',
 'with',
 'such',
 'nonsense']

In [40]:
num_features = 300
skip_gram = 1
workers = multiprocessing.cpu_count()

In [45]:
potter2vec = gensim.models.word2vec.Word2Vec(sentences, workers=workers, sg=skip_gram, iter=10)

KeyboardInterrupt: 